In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("/content/LEPRA_Foot_Meaurement_Form Revised.xlsx")

In [3]:
df.head()

,State,District,Footwear,Footwear/Leprosy,Footwear/Lymphatic Filariasis (LF),Project Name,Project Area,RC Name,Name of the Shoe technician,Locatioion,...,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,__version__,_tags,_index
0,Telangana,Nizamabad,Leprosy,1,0,NTDRU,RC,Nirmal Referral Centre,Devadas,Others,...,497311588,be0c31be-37ae-4b65-874d-f0c59f5ae9ab,2023-10-28 07:25:14,NaN,NaN,submitted_via_web,leprakobo,v9CuY2fsGCiSz4EKwBegft,NaN,338
1,Telangana,Nirmal,Leprosy,1,0,NTDRU,RC,Nirmal Referral Centre,Pothanna,Others,...,497961400,19e37c48-b246-4337-9657-35a72a918996,2023-10-30 08:27:16,NaN,NaN,submitted_via_web,leprakobo,v9CuY2fsGCiSz4EKwBegft,NaN,341
2,Telangana,Nirmal,Leprosy,1,0,NTDRU,RC,Nirmal Referral Centre,Pothanna,Others,...,498441603,ec9511c8-9fc7-4457-a76c-c51f8d521d62,2023-10-31 08:36:32,NaN,NaN,submitted_via_web,leprakobo,v9CuY2fsGCiSz4EKwBegft,NaN,343
3,Telangana,Nirmal,Leprosy,1,0,NTDRU,RC,Nirmal Referral Centre,Pothanna,Others,...,498909714,f0034dc4-75a2-4aac-8bd7-d65483c2980b,2023-11-01 08:18:51,NaN,NaN,submitted_via_web,leprakobo,v9CuY2fsGCiSz4EKwBegft,NaN,344
4,Telangana,Hyderabad,Leprosy,1,0,NTDRU,RC,Kagaz Nagar Referral Centre,Devadas,Others,...,502777843,f0edfe76-e7dd-49be-8e3d-41768a22e639,2023-11-10 07:34:29,NaN,NaN,submitted_via_web,leprakobo,v9CuY2fsGCiSz4EKwBegft,NaN,353


In [4]:
df.columns

Index(['State', 'District', 'Footwear', 'Footwear/Leprosy',
       'Footwear/Lymphatic Filariasis (LF)', 'Project Name', 'Project Area',
       'RC Name', 'Name of the Shoe technician', 'Locatioion', 'Patient Age',
       'Patient Gender', 'Grade', 'Diabeties', 'Date of the Measurement taken',
       'Length of Foot Right', 'Length of Foot Left', 'Ball Girth Right',
       'Ball Girth Left', 'In Step Right', 'In Step Left',
       'Height of Malleoli Right', 'Height of Malleoli Left',
       'Height of Malleoli Right.1', 'Height of Malleoli Left.1',
       'Foot Front View Image Right Leg',
       'Foot Front View Image Right Leg_URL', 'Foot Front View Image Left Leg',
       'Foot Front View Image Left Leg_URL', 'Foot Sole View Image Right',
       'Foot Sole View Image Right_URL', 'Foot Sole View Image Left',
       'Foot Sole View Image Left_URL', 'Notes / Remarks',
       'Date of the footwear issued', '_id', '_uuid', '_submission_time',
       '_validation_status', '_notes', '_sta

In [62]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib


# Columns to be used
input_features = ['Patient Age', 'Patient Gender', 'Diabeties', 'Grade']
output_features = [
    'Length of Foot Right', 'Length of Foot Left',
    'Ball Girth Right', 'Ball Girth Left',
    'In Step Right', 'In Step Left'
]

# Create a copy of the dataframe to avoid modifying the original df in place for clustering
df_clustering = df.copy()

# Drop rows with missing required values specifically for clustering features and output features
df_clustering = df_clustering.dropna(subset=input_features + output_features)

# Encode categorical columns for clustering and store encoders
label_encoders = {}
categorical_cols = ['Patient Gender', 'Diabeties', 'Grade'] # List of categorical columns to encode

for col in categorical_cols:
    if col in df_clustering.columns:
        le = LabelEncoder()
        # Fit the encoder on the string representation of the column *before* transforming
        le.fit(df_clustering[col].astype(str))
        label_encoders[col] = le
        # Transform the column in the clustering dataframe
        df_clustering[col] = le.transform(df_clustering[col].astype(str))
    else:
        print(f"Warning: Categorical column '{col}' not found in dataframe for clustering.")


# Split features
X = df_clustering[input_features]
y = df_clustering[output_features]

# Scale input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit KMeans
# Ensure n_init is set explicitly if using scikit-learn versions >= 1.4
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10) # Added n_init
df_clustering['Cluster'] = kmeans.fit_predict(X_scaled)

# Calculate average measurements per cluster
cluster_means = df_clustering.groupby('Cluster')[output_features].mean()

# Save models and encoders
joblib.dump(kmeans, "kmeans_model.joblib")
joblib.dump(scaler, "input_scaler.joblib")
joblib.dump(label_encoders, "label_encoders.joblib")
joblib.dump(cluster_means, "cluster_means.joblib")

print("✅ Model, scalers, encoders, and cluster means saved.")

# --- Debugging: Load and check saved encoders ---
try:
    loaded_debug_encoders = joblib.load("label_encoders.joblib")
    print("\n--- Debugging Saved Encoders ---")
    for col in categorical_cols:
        if col in loaded_debug_encoders and hasattr(loaded_debug_encoders[col], 'classes_'):
            print(f"'{col}' classes after loading: {loaded_debug_encoders[col].classes_}")
        elif col in loaded_debug_encoders:
             print(f"'{col}' encoder loaded, but no '.classes_' attribute found.")
        else:
            print(f"'{col}' encoder not found after loading.")
    print("---------------------------------")
except FileNotFoundError:
    print("\n--- Debugging Failed: label_encoders.joblib not found after saving ---")
# -------------------------------------------------

✅ Model, scalers, encoders, and cluster means saved.

--- Debugging Saved Encoders ---
'Patient Gender' classes after loading: ['0' '1']
'Diabeties' classes after loading: ['0' '1']
'Grade' classes after loading: ['0' '1']
---------------------------------


In [57]:
from sklearn.metrics import silhouette_score

# Assuming X_scaled and kmeans are available from a previous cell (e.g., ss62XLslD-9y)
# If not, you need to run the cell that performs scaling and KMeans clustering first.
try:
    score = silhouette_score(X_scaled, kmeans.labels_)
    print("Silhouette Score:", score)
except NameError:
    print("Error: 'X_scaled' or 'kmeans' not found. Please ensure the cell that performs scaling and KMeans clustering was executed.")
except Exception as e:
    print(f"An error occurred: {e}")

Silhouette Score: 0.40119527728317994


In [58]:
import joblib

# Load the KMeans model (assuming it was saved as 'kmeans_model.joblib' in a previous step)
try:
    loaded_kmeans_model = joblib.load("kmeans_model.joblib")
    print("KMeans model loaded successfully!")
    # You can now use loaded_kmeans_model for predictions or further analysis
except FileNotFoundError:
    print("Error: 'kmeans_model.joblib' not found. Please ensure the cell that saves the model was executed.")

# You can also load the scaler and label encoders if needed
try:
    loaded_scaler = joblib.load("input_scaler.joblib")
    loaded_label_encoders = joblib.load("label_encoders.joblib")
    loaded_cluster_means = joblib.load("cluster_means.joblib")
    print("Scaler, label encoders, and cluster means loaded successfully!")
except FileNotFoundError:
    print("Error: One or more saved files (scaler, label encoders, or cluster means) not found.")

KMeans model loaded successfully!
Scaler, label encoders, and cluster means loaded successfully!


In [35]:
# Step 1: Data Preparation for Classification

from sklearn.preprocessing import LabelEncoder

# Define features (X_clf) and target variable (y_clf) for classification
# Include relevant features from the original dataframe
classification_features = [
    'Patient Age',
    'Patient Gender',
    'Diabeties',
    'Grade',
    'Length of Foot Right',
    'Length of Foot Left',
    'Ball Girth Right',
    'Ball Girth Left',
    'In Step Right',
    'In Step Left'
]
classification_target = 'Footwear/Leprosy'

# Drop rows with missing values in the selected features or target
# Assuming df is already loaded and potentially modified by previous steps (e.g., clustering)
df_clf = df.dropna(subset=classification_features + [classification_target]).copy() # Use .copy() to avoid SettingWithCopyWarning

# Separate features and target
X_clf = df_clf[classification_features].copy() # Use .copy()
y_clf = df_clf[classification_target].copy() # Use .copy()


# Encode categorical features in X_clf and store encoders
categorical_clf_features = ['Patient Gender', 'Diabeties', 'Grade']
label_encoders_clf = {}

for col in categorical_clf_features:
    if col in X_clf.columns:
        # Create and fit a new LabelEncoder for each categorical column
        # This ensures the encoder is available with .classes_ regardless of prior encoding status in df
        le = LabelEncoder()
        # Fit on the values from the classification dataframe
        le.fit(df_clf[col].astype(str))
        label_encoders_clf[col] = le

        # Transform the column in X_clf using the fitted encoder
        X_clf.loc[:, col] = le.transform(X_clf[col].astype(str))


print("✅ Data prepared for classification.")
display(X_clf.head())
display(y_clf.head())

✅ Data prepared for classification.


,Patient Age,Patient Gender,Diabeties,Grade,Length of Foot Right,Length of Foot Left,Ball Girth Right,Ball Girth Left,In Step Right,In Step Left
0,50,0,1,0,23.0,23.0,20.0,20.0,23.0,23.0
1,70,1,0,1,22.0,18.0,15.0,15.0,20.0,20.0
2,35,1,0,1,26.0,26.0,14.0,14.0,18.0,18.0
3,60,1,0,1,23.0,22.0,12.0,12.0,16.0,16.0
4,75,1,0,0,28.0,28.0,26.0,24.0,26.0,28.0


,Footwear/Leprosy
0,1
1,1
2,1
3,1
4,1


In [37]:
# Step 2: Train a Classification Model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split data into training and testing sets
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42, stratify=y_clf)

# Initialize and train a classification model (e.g., Logistic Regression)
# You could also try other models like RandomForestClassifier, Support Vector Machines, etc.
classifier_model = LogisticRegression(random_state=42, max_iter=200)

classifier_model.fit(X_train_clf, y_train_clf)

# Make predictions on the test set
y_pred_clf = classifier_model.predict(X_test_clf)

# Evaluate the model
accuracy = accuracy_score(y_test_clf, y_pred_clf)
report = classification_report(y_test_clf, y_pred_clf)

print("✅ Classification model trained.")
print("\nModel Accuracy:", accuracy)
print("\nClassification Report:\n", report)

# Note: For a more robust evaluation, cross-validation should be used.

✅ Classification model trained.

Model Accuracy: 0.852112676056338

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.05      0.09        22
           1       0.85      1.00      0.92       120

    accuracy                           0.85       142
   macro avg       0.93      0.52      0.50       142
weighted avg       0.87      0.85      0.79       142



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
# Step 3: Save the Classification Model

import joblib

# Save the trained classification model
joblib.dump(classifier_model, "leprosy_classifier_model.joblib")

# Save the label encoders used for classification features
joblib.dump(label_encoders_clf, "leprosy_label_encoders.joblib")

# Save the list of feature names (column order) used during training
joblib.dump(X_clf.columns.tolist(), "leprosy_classification_features.joblib")


print("✅ Classification model, label encoders, and feature names saved.")

✅ Classification model, label encoders, and feature names saved.


In [8]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [67]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# Load saved objects for Clustering
try:
    kmeans = joblib.load("kmeans_model.joblib")
    scaler = joblib.load("input_scaler.joblib")
    label_encoders_clustering = joblib.load("label_encoders.joblib") # Renamed for clarity
    cluster_means = joblib.load("cluster_means.joblib")
    st.sidebar.success("Clustering models loaded successfully!")
except FileNotFoundError:
    st.sidebar.error("Clustering model files not found. Please check and try again.")
    # In a real Streamlit app, you might handle this more gracefully
    # st.stop() # Don't stop the app if clustering models are missing, allow classification to run if available

# Load saved objects for Classification
try:
    classifier_model = joblib.load("leprosy_classifier_model.joblib")
    label_encoders_classification = joblib.load("leprosy_label_encoders.joblib") # Load classification specific encoders
    classification_features_names = joblib.load("leprosy_classification_features.joblib") # Load feature names
    st.sidebar.success("Classification model loaded successfully!")
except FileNotFoundError:
     st.sidebar.error("Classification model files not found. Please check and try again.")
     # st.stop() # Don't stop the app if classification models are missing, allow clustering to run if available

# Define mappings for display labels and their underlying numerical values
# Assuming the numerical encoding (0, 1) corresponds to specific categories
display_mappings = {
    'Patient Gender': {0: 'Male', 1: 'Female'},
    'Diabeties': {0: 'No', 1: 'Yes'},
    'Grade': {0: 'Unknown', 1: 'Grade 1', 2: 'Grade 2'} # Updated Grade mapping for display
}

# Mapping from display label back to the numerical value for model input
# This needs to handle the new Grade options mapping back to the model's expected 0/1
display_to_numerical_mapping = {
    'Patient Gender': {'Male': 0, 'Female': 1},
    'Diabeties': {'No': 0, 'Yes': 1},
    'Grade': {'Unknown': 0, 'Grade 1': 1, 'Grade 2': 1} # Map Grade 1 and Grade 2 to the same numerical value (1) for the binary model
}


# Streamlit UI enhancements
st.set_page_config(layout="wide") # Use wide layout

st.title("Foot Measurement Estimator and Leprosy Risk Predictor")

st.markdown("""
This application has two main sections:
1.  **Foot Measurement Estimation:** Estimates foot measurements based on age, gender, diabetes, and leprosy grade using a clustering model.
2.  **Leprosy Risk Prediction:** Predicts the likelihood of having leprosy based on patient information and foot measurements using a classification model.
""")

# Add navigation or sections
st.sidebar.title("Navigation")
page = st.sidebar.radio("Go to", ["Foot Measurement Estimation", "Leprosy Risk Prediction"])


if page == "Foot Measurement Estimation":
    st.header("Foot Measurement Estimation")
    st.markdown("""
    Enter the patient's information below to get an estimated average foot measurement based on similar patient profiles.
    """)

    if 'kmeans' in locals(): # Check if clustering models were loaded
        # Create columns for better layout
        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Patient Information for Estimation")
            # Collect user input for clustering features
            age_est = st.number_input("Enter Patient Age for Estimation", min_value=0, max_value=120, value=50, key='age_est')

            # Use display_mappings for selectbox options, but get the underlying numerical value
            # Ensure keys exist in display_mappings
            gender_display_options = list(display_mappings['Patient Gender'].values()) if 'Patient Gender' in display_mappings else ['Unknown']
            diabeties_display_options = list(display_mappings['Diabeties'].values()) if 'Diabeties' in display_mappings else ['Unknown']
            grade_display_options = list(display_mappings['Grade'].values()) if 'Grade' in display_mappings else ['Unknown']

            gender_est_label = st.selectbox("Select Gender for Estimation", gender_display_options, key='gender_est_label')
            diabeties_est_label = st.selectbox("Diabeties Status for Estimation", diabeties_display_options, key='diabeties_est_label')
            grade_est_label = st.selectbox("Grade of Leprosy for Estimation", grade_display_options, key='grade_est_label')

            # Get the numerical value corresponding to the selected label using display_to_numerical_mapping
            gender_est = display_to_numerical_mapping['Patient Gender'].get(gender_est_label, 0) # Default to 0 if label not found
            diabeties_est = display_to_numerical_mapping['Diabeties'].get(diabeties_est_label, 0) # Default to 0 if label not found
            grade_est = display_to_numerical_mapping['Grade'].get(grade_est_label, 0) # Default to 0 if label not found


        with col2:
            st.subheader("Estimation Results")
            if st.button("Estimate Foot Measurements"):
                try:
                    # Create input DataFrame using the numerical values
                    input_df_est = pd.DataFrame({
                        'Patient Age': [age_est],
                        'Patient Gender': [gender_est],
                        'Diabeties': [diabeties_est],
                        'Grade': [grade_est]
                    })

                    # Encode categorical features using clustering encoders (will encode 0/1 strings to 0/1 integers if needed)
                    for col in ['Patient Gender', 'Diabeties', 'Grade']:
                         if col in input_df_est.columns and col in label_encoders_clustering:
                            le = label_encoders_clustering[col]
                            # Transform the numerical value (as string) using the loaded encoder
                            # Check if the numerical value as string is in the encoder's classes
                            if str(input_df_est[col].iloc[0]) in le.classes_:
                                input_df_est[col] = le.transform(input_df_est[col].astype(str))
                            else:
                                st.warning(f"Numerical value '{input_df_est[col].iloc[0]}' for '{col}' is not recognized by clustering encoder. Using a default encoding (0).")
                                # Handle unseen data - use a default. Using 0 as a simple default.
                                input_df_est[col] = 0
                         elif col in input_df_est.columns:
                             # If encoder not found for this column, assume it should be numerical and cast
                             input_df_est[col] = input_df_est[col].astype(int)


                    # Scale inputs using clustering scaler
                    input_scaled_est = scaler.transform(input_df_est)

                    # Predict cluster
                    cluster = kmeans.predict(input_scaled_est)[0]

                    st.info(f"Predicted Cluster: {cluster}")
                    st.subheader("Estimated Average Foot Measurements:")

                    cluster_output = cluster_means.loc[cluster][[
                        'Length of Foot Right',
                        'Length of Foot Left',
                        'Ball Girth Right',
                        'Ball Girth Left',
                        'In Step Right',
                        'In Step Left'
                    ]]

                    st.dataframe(cluster_output.to_frame(name="Average Value"))

                except Exception as e:
                    st.error(f"Estimation failed: {e}")
    else:
        st.warning("Foot measurement estimation models are not available. Please ensure all model files are uploaded.")


elif page == "Leprosy Risk Prediction":
    st.header("Leprosy Risk Prediction")
    st.markdown("""
    Enter the patient's information and foot measurements below to get a prediction on the likelihood of having leprosy.
    """)

    if 'classifier_model' in locals() and 'classification_features_names' in locals(): # Check if classification models and feature names were loaded
        # Collect user input for classification features
        st.subheader("Patient Information and Foot Measurements for Prediction")
        age_pred = st.number_input("Enter Patient Age for Prediction", min_value=0, max_value=120, value=50, key='age_pred')

        # Use display_mappings for selectbox options in prediction section as well
        gender_pred_display_options = list(display_mappings['Patient Gender'].values()) if 'Patient Gender' in display_mappings else ['Unknown']
        diabeties_pred_display_options = list(display_mappings['Diabeties'].values()) if 'Diabeties' in display_mappings else ['Unknown']
        grade_pred_display_options = list(display_mappings['Grade'].values()) if 'Grade' in display_mappings else ['Unknown']


        gender_pred_label = st.selectbox("Select Gender for Prediction", gender_pred_display_options, key='gender_pred_label')
        diabeties_pred_label = st.selectbox("Diabeties Status for Prediction", diabeties_pred_display_options, key='diabeties_pred_label')
        grade_pred_label = st.selectbox("Grade of Leprosy for Prediction", grade_pred_display_options, key='grade_pred_label')

        # Get the numerical value corresponding to the selected label using display_to_numerical_mapping
        gender_pred = display_to_numerical_mapping['Patient Gender'].get(gender_pred_label, 0) # Default to 0 if label not found
        diabeties_pred = display_to_numerical_mapping['Diabeties'].get(diabeties_pred_label, 0) # Default to 0 if label not found
        grade_pred = display_to_numerical_mapping['Grade'].get(grade_pred_label, 0) # Default to 0 if label not found


        st.markdown("---")
        st.subheader("Foot Measurements")
        length_right = st.number_input("Length of Foot Right (cm)", min_value=0.0, value=25.0, key='length_right')
        length_left = st.number_input("Length of Foot Left (cm)", min_value=0.0, value=25.0, key='length_left')
        ball_girth_right = st.number_input("Ball Girth Right (cm)", min_value=0.0, value=22.0, key='ball_girth_right')
        ball_girth_left = st.number_input("Ball Girth Left (cm)", min_value=0.0, value=22.0, key='ball_girth_left')
        in_step_right = st.number_input("In Step Right (cm)", min_value=0.0, value=24.0, key='in_step_right')
        in_step_left = st.number_input("In Step Left (cm)", min_value=0.0, value=24.0, key='in_step_left')


        if st.button("Predict Leprosy Risk"):
            try:
                # Create input DataFrame for classification using numerical values
                input_df_pred = pd.DataFrame({
                    'Patient Age': [age_pred],
                    'Patient Gender': [gender_pred],
                    'Diabeties': [diabeties_pred],
                    'Grade': [grade_pred],
                    'Length of Foot Right': [length_right],
                    'Length of Foot Left': [length_left],
                    'Ball Girth Right': [ball_girth_right],
                    'Ball Girth Left': [ball_girth_left],
                    'In Step Right': [in_step_right],
                    'In Step Left': [in_step_left]
                })

                # Encode categorical features using classification encoders (will encode 0/1 strings to 0/1 integers if needed)
                for col in ['Patient Gender', 'Diabeties', 'Grade']:
                     if col in input_df_pred.columns and col in label_encoders_classification:
                        le = label_encoders_classification[col]
                         # Transform the numerical value (as string) using the loaded encoder
                        # Check if the numerical value as string is in the encoder's classes
                        if str(input_df_pred[col].iloc[0]) in le.classes_:
                             input_df_pred[col] = le.transform(input_df_pred[col].astype(str))
                        else:
                             st.warning(f"Numerical value '{input_df_pred[col].iloc[0]}' for '{col}' is not recognized by classification encoder. Using a default encoding (0).")
                             # Handle unseen data - use a default. Using 0 as a simple default.
                             input_df_pred[col] = 0

                     elif col in input_df_pred.columns:
                         # If encoder not found for this column, assume it should be numerical and cast
                         input_df_pred[col] = input_df_pred[col].astype(int)


                # Ensure column order matches training data using loaded feature names
                if classification_features_names:
                     # Reindex to match training columns order
                     input_df_pred = input_df_pred[classification_features_names]
                else:
                     st.warning("Classification training feature names not found. Proceeding with input order. Ensure consistency.")


                prediction_proba = classifier_model.predict_proba(input_df_pred)[0]
                # Assuming the target variable 'Footwear/Leprosy' is binary (0 or 1)
                # We need to know which class index corresponds to 'Leprosy' (1)
                # Based on the classification report, class 1 is Leprosy
                leprosy_probability = prediction_proba[1] # Probability of class 1 (Leprosy)
                no_leprosy_probability = prediction_proba[0] # Probability of class 0 (No Leprosy)

                st.subheader("Prediction Results:")
                # Display probability as percentage
                st.info(f"Predicted Probability of having Leprosy: **{leprosy_probability:.2%}**")

                # Create Pie Chart
                labels = ['Probability of Leprosy', 'Probability of No Leprosy']
                sizes = [leprosy_probability, no_leprosy_probability]
                colors = ['#ff9999','#66b3ff'] # Example colors
                explode = (0.1, 0)  # explode 1st slice (i.e. 'Probability of Leprosy')

                fig1, ax1 = plt.subplots()
                ax1.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
                        shadow=True, startangle=90)
                ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

                st.pyplot(fig1)

            except Exception as e:
                st.error(f"Prediction failed: {e}")
    else:
        st.warning("Leprosy risk prediction models or feature names are not available. Please ensure all model files are uploaded.")

st.markdown("---") # Add a horizontal rule
st.markdown("© 2024 Foot Measurement Estimator and Leprosy Risk Predictor")

Overwriting app.py


In [68]:
!streamlit run app.py &>/content/logs.txt &

In [44]:
from pyngrok import ngrok
ngrok.set_auth_token("2tqLkAmewm6lJ8FR1w7QDzyLPYF_5ym1o2fHCn9tP6a2ibLQT")

In [69]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"streamlit App is live at: {public_url}")

streamlit App is live at: NgrokTunnel: "https://ea97f0991ac4.ngrok-free.app" -> "http://localhost:8501"


In [66]:
!pkill -f ngrok

In [70]:
%%writefile requirements.txt
pandas
numpy
scikit-learn
streamlit
joblib
matplotlib
seaborn

Overwriting requirements.txt
